# Try ELG Annotator

In [19]:
from gatenlp import Document, Span
from gatenlp.processing.client import ElgTextAnnotator
from elg import Authentication

In [20]:

# txt = "A 💩 word.\nBarack Obama met Angela Merkel in New York."
txt = "Ein 💩 Wort.\nIch habs ins Dings gegeben."
doc = Document(txt)
doc

Document(Ein 💩 Wort.
Ich habs ins Dings gegeben.,features=Features({}),anns={})

In [21]:
# Only run if required: first time or when the tokens have expired
#auth = Authentication.init(scope="offline_access")
#auth.to_json("tokens.json")

#print(f"\n\nThe tokens will expire the {auth.refresh_expires_time}")

In [37]:
# if we run the service locally in docker
# annt = ElgTextAnnotator(url="http://localhost:8801/udpipe/en")

# annt = ElgTextAnnotator(
#     url="https://live.european-language-grid.eu/execution/process/gatenercen",
#     auth_file="tokens.json"
# )
doc.annset().clear()
annt = ElgTextAnnotator(
    url="https://live.european-language-grid.eu/execution/process/udpipede",
    auth_file="tokens.json", 
    anntypes_map = {"udpipe/paragraphs": "Paragraph", "udpipe/sentences": "Sentence", "udpipe/tokens": "UDPToken"}
)

In [38]:
annt(doc)

Document(Ein 💩 Wort.
Ich habs ins Dings gegeben.,features=Features({}),anns={'': AnnotationSet([Annotation(0,39,Paragraph,features=Features({}),id=43), Annotation(0,11,Sentence,features=Features({}),id=44), Annotation(0,3,UDPToken,features=Features({'words': [{'form': 'Ein', 'lemma': 'ein', 'upos': 'DET', 'xpos': 'ART', 'feats': 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|PronType=Art', 'head': 3, 'deprel': 'det'}]}),id=46), Annotation(4,5,UDPToken,features=Features({'words': [{'form': '💩', 'lemma': '💩', 'upos': 'SYM', 'xpos': 'NE', 'head': 3, 'deprel': 'cc'}]}),id=47), Annotation(6,10,UDPToken,features=Features({'words': [{'form': 'Wort', 'lemma': 'Wort', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Case=Nom|Gender=Neut|Number=Sing', 'head': 0, 'deprel': 'root'}]}),id=48), Annotation(10,11,UDPToken,features=Features({'words': [{'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '$.', 'head': 3, 'deprel': 'punct'}]}),id=49), Annotation(12,39,Sentence,features=Features({}),id=45), Annotation(12,15,UDPToken,features=Features({'words': [{'form': 'Ich', 'lemma': 'ich', 'upos': 'PRON', 'xpos': 'PPER', 'feats': 'Case=Nom|Number=Sing|Person=1|PronType=Prs', 'head': 6, 'deprel': 'nsubj'}]}),id=50), Annotation(16,20,UDPToken,features=Features({'words': [{'form': 'habs', 'lemma': 'habs', 'upos': 'X', 'xpos': 'NN', 'feats': 'Case=Acc|Gender=Masc|Number=Plur', 'head': 6, 'deprel': 'obj'}]}),id=51), Annotation(21,24,UDPToken,features=Features({'words': [{'form': 'in', 'lemma': 'in', 'upos': 'ADP', 'xpos': 'APPR', 'head': 5, 'deprel': 'case'}, {'form': 'das', 'lemma': 'der', 'upos': 'DET', 'xpos': 'ART', 'feats': 'Case=Acc|Definite=Def|Gender=Neut|Number=Sing|PronType=Art', 'head': 5, 'deprel': 'det'}]}),id=52), Annotation(25,30,UDPToken,features=Features({'words': [{'form': 'Dings', 'lemma': 'Dings', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Case=Acc|Gender=Neut|Number=Plur', 'head': 6, 'deprel': 'obl'}]}),id=53), Annotation(31,38,UDPToken,features=Features({'words': [{'form': 'gegeben', 'lemma': 'geben', 'upos': 'VERB', 'xpos': 'VVPP', 'feats': 'VerbForm=Part', 'head': 0, 'deprel': 'root'}]}),id=54), Annotation(38,39,UDPToken,features=Features({'words': [{'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '$.', 'head': 6, 'deprel': 'punct'}]}),id=55)])})

In [39]:
doc[2:3]

'n'

In [40]:
# convert the UDPToken annotations into actual Token, Word and MWT annotations, as is done for Stanza
# for now, do not bother with the dependency parse ids
# If an UDPToken contains a single word, we generate a Token annotation
# If there are several, we generate a MWT annotation over the span and for each word a Token annotation, 
#   where the spans of the token annotations get calculated using Span.squeeze(start, end, nrwords)
utokens = doc.annset().with_type("UDPToken")
outset = doc.annset()
for utoken in utokens:
    words = utoken.features["words"]
    if len(words) == 1:
        outset.add(utoken.start, utoken.end, "Token", features=words[0])
    else:
        spans = Span.squeeze(utoken.start, utoken.end, len(words))
        assert len(words)==len(spans)
        annids = []
        for word, span in zip(words,spans):
            ann = outset.add(span.start, span.end, "Token", features=word)
            annids.append(ann.id)
        outset.add(utoken.start, utoken.end, "MWT", features=dict(word_ids=annids))

In [41]:
doc

Document(Ein 💩 Wort.
Ich habs ins Dings gegeben.,features=Features({}),anns={'': AnnotationSet([Annotation(0,39,Paragraph,features=Features({}),id=43), Annotation(0,11,Sentence,features=Features({}),id=44), Annotation(0,3,UDPToken,features=Features({'words': [{'form': 'Ein', 'lemma': 'ein', 'upos': 'DET', 'xpos': 'ART', 'feats': 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|PronType=Art', 'head': 3, 'deprel': 'det'}]}),id=46), Annotation(0,3,Token,features=Features({'form': 'Ein', 'lemma': 'ein', 'upos': 'DET', 'xpos': 'ART', 'feats': 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|PronType=Art', 'head': 3, 'deprel': 'det'}),id=56), Annotation(4,5,UDPToken,features=Features({'words': [{'form': '💩', 'lemma': '💩', 'upos': 'SYM', 'xpos': 'NE', 'head': 3, 'deprel': 'cc'}]}),id=47), Annotation(4,5,Token,features=Features({'form': '💩', 'lemma': '💩', 'upos': 'SYM', 'xpos': 'NE', 'head': 3, 'deprel': 'cc'}),id=57), Annotation(6,10,UDPToken,features=Features({'words': [{'form': 'Wort', 'lemma': 'Wort', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Case=Nom|Gender=Neut|Number=Sing', 'head': 0, 'deprel': 'root'}]}),id=48), Annotation(6,10,Token,features=Features({'form': 'Wort', 'lemma': 'Wort', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Case=Nom|Gender=Neut|Number=Sing', 'head': 0, 'deprel': 'root'}),id=58), Annotation(10,11,UDPToken,features=Features({'words': [{'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '$.', 'head': 3, 'deprel': 'punct'}]}),id=49), Annotation(10,11,Token,features=Features({'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '$.', 'head': 3, 'deprel': 'punct'}),id=59), Annotation(12,39,Sentence,features=Features({}),id=45), Annotation(12,15,UDPToken,features=Features({'words': [{'form': 'Ich', 'lemma': 'ich', 'upos': 'PRON', 'xpos': 'PPER', 'feats': 'Case=Nom|Number=Sing|Person=1|PronType=Prs', 'head': 6, 'deprel': 'nsubj'}]}),id=50), Annotation(12,15,Token,features=Features({'form': 'Ich', 'lemma': 'ich', 'upos': 'PRON', 'xpos': 'PPER', 'feats': 'Case=Nom|Number=Sing|Person=1|PronType=Prs', 'head': 6, 'deprel': 'nsubj'}),id=60), Annotation(16,20,UDPToken,features=Features({'words': [{'form': 'habs', 'lemma': 'habs', 'upos': 'X', 'xpos': 'NN', 'feats': 'Case=Acc|Gender=Masc|Number=Plur', 'head': 6, 'deprel': 'obj'}]}),id=51), Annotation(16,20,Token,features=Features({'form': 'habs', 'lemma': 'habs', 'upos': 'X', 'xpos': 'NN', 'feats': 'Case=Acc|Gender=Masc|Number=Plur', 'head': 6, 'deprel': 'obj'}),id=61), Annotation(21,24,UDPToken,features=Features({'words': [{'form': 'in', 'lemma': 'in', 'upos': 'ADP', 'xpos': 'APPR', 'head': 5, 'deprel': 'case'}, {'form': 'das', 'lemma': 'der', 'upos': 'DET', 'xpos': 'ART', 'feats': 'Case=Acc|Definite=Def|Gender=Neut|Number=Sing|PronType=Art', 'head': 5, 'deprel': 'det'}]}),id=52), Annotation(21,23,Token,features=Features({'form': 'in', 'lemma': 'in', 'upos': 'ADP', 'xpos': 'APPR', 'head': 5, 'deprel': 'case'}),id=62), Annotation(21,24,MWT,features=Features({'word_ids': [62, 63]}),id=64), Annotation(23,24,Token,features=Features({'form': 'das', 'lemma': 'der', 'upos': 'DET', 'xpos': 'ART', 'feats': 'Case=Acc|Definite=Def|Gender=Neut|Number=Sing|PronType=Art', 'head': 5, 'deprel': 'det'}),id=63), Annotation(25,30,UDPToken,features=Features({'words': [{'form': 'Dings', 'lemma': 'Dings', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Case=Acc|Gender=Neut|Number=Plur', 'head': 6, 'deprel': 'obl'}]}),id=53), Annotation(25,30,Token,features=Features({'form': 'Dings', 'lemma': 'Dings', 'upos': 'NOUN', 'xpos': 'NN', 'feats': 'Case=Acc|Gender=Neut|Number=Plur', 'head': 6, 'deprel': 'obl'}),id=65), Annotation(31,38,UDPToken,features=Features({'words': [{'form': 'gegeben', 'lemma': 'geben', 'upos': 'VERB', 'xpos': 'VVPP', 'feats': 'VerbForm=Part', 'head': 0, 'deprel': 'root'}]}),id=54), Annotation(31,38,Token,features=Features({'form': 'gegeben', 'lemma': 'geben', 'upos': 'VERB', 'xpos': 'VVPP', 'feats': 'VerbForm=Part', 'head': 0, 'deprel': 'root'}),id=66), Annotation(38,39,UDPToken,features=Featur

In [34]:
anns = doc.annset().with_type("MWT")

In [35]:
for ann in anns:
    print(ann.features)

Features({'word_ids': [Annotation(21,23,Token,features=Features({'form': 'in', 'lemma': 'in', 'upos': 'ADP', 'xpos': 'APPR', 'head': 5, 'deprel': 'case'}),id=25), Annotation(23,24,Token,features=Features({'form': 'das', 'lemma': 'der', 'upos': 'DET', 'xpos': 'ART', 'feats': 'Case=Acc|Definite=Def|Gender=Neut|Number=Sing|PronType=Art', 'head': 5, 'deprel': 'det'}),id=26)]})
Features({'word_ids': [37, 38]})
